## 데이터 품질 전처리
데이터 품질 전처리에 필요한 6가지 품질 평가지표(완전성, 유일성, 유효성, 일관성, 정확성, 무결성)의 산출 방법에 대하여 기술한다.

In [36]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
from dateutil.parser import parse
import math
# 경로 설정은 각자 데이터가 저장된 디렉토리에 맞게 해줍니다.
chg_file_paths = [f'../../../../data/raw_data/train/{num}_chg.csv' for num in range(1000,1051)]
dchg_file_paths = [f'../../../../data/raw_data/train/{num}_dchg.csv' for num in range(1000,1051)]

In [29]:
chg_datas = [pd.read_csv(file_path) for file_path in chg_file_paths]
dchg_datas = [pd.read_csv(file_path) for file_path in dchg_file_paths]

▶ 품질 전처리를 위해 주어진 문자열이 date, time 형식인지 체크하는 함수를 정의한다.

In [6]:
# 문자열이 date인지 확인
def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False
# 문자열이 time인지 확인
def is_time(string):
    try:
        datetime.strptime(string, '%H:%M:%S')
        return True
    except ValueError:
        return False

▶ 완전성 품질지수
$$
   완전성 품질지수 = \left(1 - \frac{\text{결측데이터의 개수}}{\text{전체 데이터의 개수}}\right) \times 100
$$  
① 총 결측 데이터의 개수를 확인하기 위해 isnull() 함수를 사용하여 각 열에서의 결측 데이터의 개수를 구한 뒤 sum() 함수를 이용하여 합산한다. 다음 함수 getCompleteness()는 완전성 품질 지수를 구하는 함수이다.

In [30]:
# 완전성 품질 지수를 구하는 함수
# 전체 결측 값의 수와 품질 지수(%)를 반환
def getCompleteness(data):
    total_null_values = data.isnull().sum().sum()
    total_notNull_values = data.count().sum()
    total_values = total_null_values + total_notNull_values
    completeness = (total_notNull_values/total_values)*100
    print(f"결측 데이터의 개수 : {total_null_values}개")
    print(f'전체 데이터의 수 : {total_values}개')
    print(f"완전성 품질 지수 : {completeness: .2f}%")
    return total_null_values, completeness

In [32]:
for data in chg_datas:
    print('='*50)
    print('일련번호 : ', data['SerialNumber'][0])
    getCompleteness(data)

일련번호 :  1000
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1388079개
완전성 품질 지수 :  100.00%
일련번호 :  1001
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1388079개
완전성 품질 지수 :  100.00%
일련번호 :  1002
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1387848개
완전성 품질 지수 :  100.00%
일련번호 :  1003
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1387848개
완전성 품질 지수 :  100.00%
일련번호 :  1004
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1966965개
완전성 품질 지수 :  100.00%
일련번호 :  1005
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1966734개
완전성 품질 지수 :  100.00%
일련번호 :  1006
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1107876개
완전성 품질 지수 :  100.00%
일련번호 :  1007
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1107645개
완전성 품질 지수 :  100.00%
일련번호 :  1008
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1398705개
완전성 품질 지수 :  100.00%
일련번호 :  1009
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1397319개
완전성 품질 지수 :  100.00%
일련번호 :  1010
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1407252개
완전성 품질 지수 :  100.00%
일련번호 :  1011
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1407483개
완전성 품질 지수 :  100.00%
일련번호 :  1012
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1150842개
완전성 품질 지수 :  100.00%
일련번호 :  1013
결측 데이터의 개수 : 59개
전체 데이터의 수 : 1903440개
완전성 품질 지수 :  100.00%
일련번호 

② 위 함수의 출력인 완전성 품질 지수는 대부분의 데이터가 100%로(최저 99.95%) 완전성 품질 기준을 만족하는 것을 확인할 수 있다. 하지만 일부 결측치가 존재하는 데이터에 대해 다음의 결측치 처리함수인 handleMissingValue()를 사용하여 결측 데이터를 처리한 후, 다시 함수 getCompleteness()를 호출하여 완전성 품질 지수를 구한다.

In [37]:
# 결측 데이터 처리 함수
def handleMissingValue(data):
    df = data.copy()
    numeric_df = df.columns[df.dtypes !='object']
    categorical_df = df.columns[df.dtypes=='object']
    # 범주형 데이터 결측 처리
    for i in categorical_df:
        df[i].fillna(df[i].mode()[0], inplace = True)
    
    # 결측치로만 이루어진 열 제거
    for col in df.columns:
        if df[col].isnull().all():
            df = df.drop([col], axis = 1)
    print(df.columns)
    df_flag_null = df.isnull()
    i, c = np.where(df_flag_null)
    
    # 수치형 데이터 결측 처리
    for j in range(len(i)):
        # 첫 데이터가 결측인 경우
        if i[j]==0:
            s = df.iloc[:, c[j]]
            id_s = s.notna().idxmax()
            val = df.iat[id_s, c[j]]
            df.iat[i[j], c[j]] = val
    
        # 만약 마지막 데이터가 결측인 경우
        elif i[j] == (len(df)-1):
            print('end!')
            print(i[j], c[j])
            s = df.iloc[:, c[j]]
            id_s = s.not_na()[::-1].idxmax()
            print(id_s, c[j])
            val = df.iat[id_s, c[j]]
            print(val)
            df.iat[id_s, c[j]]
        # 가운데 데이터가 결측인 경우
        else:
            low = df.iat[i[j]-1, c[j]]
            high = df.iat[i[j]+1, c[j]]
            if math.isnan(high):
                val = low
            else:
                val = (low+high) / 2
                df.iat[i[j], c[j]] = val
        
    return df

In [38]:
for i in range(len(chg_datas)):
    print('='*50)
    print('일련번호 : ', chg_datas[i]['SerialNumber'][0])
    chg_datas[i] = handleMissingValue(chg_datas[i])

일련번호 :  1000
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg', 'USOCmin', 'USOCmax',
       ...
       'M12T01', 'M12T02', 'M13T01', 'M13T02', 'M14T01', 'M14T02', 'M15T01',
       'M15T02', 'M16T01', 'M16T02'],
      dtype='object', length=231)
일련번호 :  1001
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg', 'USOCmin', 'USOCmax',
       ...
       'M12T01', 'M12T02', 'M13T01', 'M13T02', 'M14T01', 'M14T02', 'M15T01',
       'M15T02', 'M16T01', 'M16T02'],
      dtype='object', length=231)
일련번호 :  1002
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg', 'USOCmin', 'USOCmax',
       ...
       'M12T01', 'M12T02', 'M13T01', 'M13T02', 'M14T01', 'M14T02', 'M15T01',
       'M15T02', 'M16T01', 'M16T02'],
      dtype='object', length=231)
일련번호 :  1003
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg'

일련번호 :  1028
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg', 'USOCmin', 'USOCmax',
       ...
       'M12T01', 'M12T02', 'M13T01', 'M13T02', 'M14T01', 'M14T02', 'M15T01',
       'M15T02', 'M16T01', 'M16T02'],
      dtype='object', length=231)
일련번호 :  1029
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg', 'USOCmin', 'USOCmax',
       ...
       'M12T01', 'M12T02', 'M13T01', 'M13T02', 'M14T01', 'M14T02', 'M15T01',
       'M15T02', 'M16T01', 'M16T02'],
      dtype='object', length=231)
일련번호 :  1030
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg', 'USOCmin', 'USOCmax',
       ...
       'M12T01', 'M12T02', 'M13T01', 'M13T02', 'M14T01', 'M14T02', 'M15T01',
       'M15T02', 'M16T01', 'M16T02'],
      dtype='object', length=231)
일련번호 :  1031
Index(['Date', 'Time', 'SerialNumber', 'Voltage', 'Current', 'RSOCmin',
       'RSOCmax', 'RSOCavg'

In [39]:
for data in chg_datas:
    print('='*50)
    print('일련번호 : ', data['SerialNumber'][0])
    getCompleteness(data)

일련번호 :  1000
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1388079개
완전성 품질 지수 :  100.00%
일련번호 :  1001
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1388079개
완전성 품질 지수 :  100.00%
일련번호 :  1002
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1387848개
완전성 품질 지수 :  100.00%
일련번호 :  1003
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1387848개
완전성 품질 지수 :  100.00%
일련번호 :  1004
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1966965개
완전성 품질 지수 :  100.00%
일련번호 :  1005
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1966734개
완전성 품질 지수 :  100.00%
일련번호 :  1006
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1107876개
완전성 품질 지수 :  100.00%
일련번호 :  1007
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1107645개
완전성 품질 지수 :  100.00%
일련번호 :  1008
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1398705개
완전성 품질 지수 :  100.00%
일련번호 :  1009
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1397319개
완전성 품질 지수 :  100.00%
일련번호 :  1010
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1407252개
완전성 품질 지수 :  100.00%
일련번호 :  1011
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1407483개
완전성 품질 지수 :  100.00%
일련번호 :  1012
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1150842개
완전성 품질 지수 :  100.00%
일련번호 :  1013
결측 데이터의 개수 : 0개
전체 데이터의 수 : 1903440개
완전성 품질 지수 :  100.00%
일련번호 :

결측 처리를 진행한 후 완전성 품질 지수를 구한 결과 모든 데이터의 완전성 품질 지수가 100%, 결측 데이터의 개수가 0으로 완전성 품질 기준을 만족한 것을 확인할 수 있다.

▶ 유일성 품질 지수
$$
    유일성 품질지수 = \left(\frac{\text{유일한데이터의 개수}}{\text{전체 데이터의 개수}}\right) \times 100
$$
① 데이터에서 유일한 값을 갖는 열을 찾는다. 시계열 데이터는 주로 TimeStamp 이다.  
② 이 테이블에서는 Date 및 Time 칼럼이 고유 식별자로 사용되었으며 Date 및 Time 칼럼에 중복되는 항목이 없으므로 유일성의 값은 100%로 계산된다.  
③ 다음 함수 getUniqueness()는 유일성을 구하는 함수이다.

In [41]:
# 유일성 품질 지수를 구하는 함수
def getUniqueness(data):
    total_data = data.count()
    unique = len(data.unique())
    uniqueness = (unique / total_data)*100
    return uniqueness

for data in chg_datas:
    print('='*50)
    print('일련번호 : ', data['SerialNumber'][0])
    time_series = data['Time']
    print(f"유일성 품질 지수 : {getUniqueness(time_series)}%")

일련번호 :  1000
유일성 품질 지수 : 100.0%
일련번호 :  1001
유일성 품질 지수 : 100.0%
일련번호 :  1002
유일성 품질 지수 : 100.0%
일련번호 :  1003
유일성 품질 지수 : 100.0%
일련번호 :  1004
유일성 품질 지수 : 100.0%
일련번호 :  1005
유일성 품질 지수 : 100.0%
일련번호 :  1006
유일성 품질 지수 : 100.0%
일련번호 :  1007
유일성 품질 지수 : 100.0%
일련번호 :  1008
유일성 품질 지수 : 100.0%
일련번호 :  1009
유일성 품질 지수 : 100.0%
일련번호 :  1010
유일성 품질 지수 : 100.0%
일련번호 :  1011
유일성 품질 지수 : 100.0%
일련번호 :  1012
유일성 품질 지수 : 100.0%
일련번호 :  1013
유일성 품질 지수 : 100.0%
일련번호 :  1014
유일성 품질 지수 : 100.0%
일련번호 :  1015
유일성 품질 지수 : 100.0%
일련번호 :  1016
유일성 품질 지수 : 100.0%
일련번호 :  1017
유일성 품질 지수 : 100.0%
일련번호 :  1018
유일성 품질 지수 : 100.0%
일련번호 :  1019
유일성 품질 지수 : 100.0%
일련번호 :  1020
유일성 품질 지수 : 100.0%
일련번호 :  1021
유일성 품질 지수 : 100.0%
일련번호 :  1022
유일성 품질 지수 : 100.0%
일련번호 :  1023
유일성 품질 지수 : 100.0%
일련번호 :  1024
유일성 품질 지수 : 100.0%
일련번호 :  1025
유일성 품질 지수 : 100.0%
일련번호 :  1026
유일성 품질 지수 : 100.0%
일련번호 :  1027
유일성 품질 지수 : 100.0%
일련번호 :  1028
유일성 품질 지수 : 100.0%
일련번호 :  1029
유일성 품질 지수 : 100.0%
일련번호 :  1030
유일성 품질 지수 : 100.0%
일련번호 :  

- 유일성 품질 지수 검사 결과 모든 데이터에서 유일성의 값은 100%로 계산된다.

▶ 유효성 품질 지수
$$
    유효성 품질지수 = \left(\frac{\text{유효성을 만족한 데이터의 개수}}{\text{전체 데이터의 개수}}\right) \times 100
$$
① 데이터가 유효 범위를 벗어나는지를 확인한다.  
② 데이터의 유효 범위를 range_min 과 range_max 값으로 지정하며, 유효한 범위 내에 속하는 데이터 항목을 count 하여 유효성을 계산한다.  
③ 다음 함수 getValidity()는 유효성을 계산하는 함수이다.

In [42]:
# 유효성을 계산하는 함수
def getSeriesValidity(data, range_min, range_max):
    column_name = data.name
    total_data = data.count()
    if(len(column_name) >=4 and column_name[0:4] =='Time'):
        time_conv = pd.to_datetime(data).dt.time
        temp = data.loc[time_conv >= range_min]
        temp = data.loc[time_conv <= range_max]
        in_range_data = temp.count()
    elif(len(column_name) >=4 and column_name[0:4] =='Date'):
        date_conv = pd.to_datetime(data).dt.date
        temp = data.loc[date_conv >= range_min]
        temp = data.loc[date_conv <= range_max]
        in_range_data = temp.count()
    else:
        in_range_data = data.loc[(data >= range_min) & (data <= range_max)].count()
    validity = in_range_data / total_data *100
    return validity

def getValidity(data, ranges):
    validities = []
    for col in ranges:
        current_validity = getSeriesValidity(data[col], ranges[col][0], ranges[col][1])
        #print(col + " validity is: %.2f%%"%current_validity)
        validities.append(current_validity)
    average_validity = sum(validities) /len(validities)
    return average_validity

In [43]:
# Calculate the validity of all columns.
for data in chg_datas:
    columns = data.columns
    range_dictionary = {}
    default_range = (-200, 200)
    # If there is a special range to be checked, please add in this list.
    # 형식: 변수명 (범위 최소, 범위 최대)
    # 데이터 유효 범위 지정
    set_range = {
        "Voltage": (0, 1000),
        "Date": (date(1999, 1, 1), date(2023, 12, 12)),
        "Time": (time(0, 0), time(23, 59)),
        "SerialNumber": (0, 1000)
    }

    for column_name in columns:
        if column_name in set_range:
            range_dictionary[column_name] = set_range[column_name]
        else:
            range_dictionary[column_name] = default_range
    
    print('='*50)
    print('일련번호 : ', data['SerialNumber'][0])
    print(f"유효성 품질지수 : {getValidity(data, range_dictionary):.2f}")

일련번호 :  1000
유효성 품질지수 : 100.00
일련번호 :  1001
유효성 품질지수 : 99.57
일련번호 :  1002
유효성 품질지수 : 99.57
일련번호 :  1003
유효성 품질지수 : 99.57
일련번호 :  1004
유효성 품질지수 : 99.57
일련번호 :  1005
유효성 품질지수 : 99.57
일련번호 :  1006
유효성 품질지수 : 99.57
일련번호 :  1007
유효성 품질지수 : 99.57
일련번호 :  1008
유효성 품질지수 : 99.57
일련번호 :  1009
유효성 품질지수 : 99.57
일련번호 :  1010
유효성 품질지수 : 99.57
일련번호 :  1011
유효성 품질지수 : 99.57
일련번호 :  1012
유효성 품질지수 : 99.57
일련번호 :  1013
유효성 품질지수 : 99.57
일련번호 :  1014
유효성 품질지수 : 99.57
일련번호 :  1015
유효성 품질지수 : 99.57
일련번호 :  1016
유효성 품질지수 : 99.57
일련번호 :  1017
유효성 품질지수 : 99.57
일련번호 :  1018
유효성 품질지수 : 99.57
일련번호 :  1019
유효성 품질지수 : 99.57
일련번호 :  1020
유효성 품질지수 : 99.57
일련번호 :  1021
유효성 품질지수 : 99.57
일련번호 :  1022
유효성 품질지수 : 99.57
일련번호 :  1023
유효성 품질지수 : 99.57
일련번호 :  1024
유효성 품질지수 : 99.57
일련번호 :  1025
유효성 품질지수 : 99.57
일련번호 :  1026
유효성 품질지수 : 99.57
일련번호 :  1027
유효성 품질지수 : 99.57
일련번호 :  1028
유효성 품질지수 : 99.57
일련번호 :  1029
유효성 품질지수 : 99.57
일련번호 :  1030
유효성 품질지수 : 99.57
일련번호 :  1031
유효성 품질지수 : 99.57
일련번호 :  1032
유효성 품질지수 : 99.57
일련번호 :  1

- 유효성 품질 지수는 모든 데이터에서 100% 혹은 99.57%로 유효성을 만족하는 것을 확인할 수 있다.

▶ 일관성 품질 지수
$$
    일관성 품질지수 = \left(\frac{\text{일관성을 만족한 데이터의 개수}}{\text{전체 데이터의 개수}}\right) \times 100
$$
① 데이터의 타입이 정해진 형식에 맞는지 확인한다.  
② 전체 데이터의 개수와 형식에 맞는 데이터의 개수를 구하여 일관성 품질 지수를 계산한다.  
③ 다음 함수 getConsistency()는 일관성을 구하는 함수이다.

In [46]:
def getSeriesConsistency(data, data_type):
    total_data = data.count()
    #print(data_type)
    if(data.name =='Date'):
        types = list(map(lambda item: is_date(item), data))
    elif(data.name =='Time'):
        types = list(map(lambda item: is_time(item), data))
    else:
        types = list(map(lambda item: isinstance(item, data_type), data))
    types = pd.Series(types)
    consistent_data = types.sum()
    consistency = (consistent_data / total_data) *100
    return consistency
def getConsistency(data, types):
    consistent_data = []
    for col in types:
        current_const = getSeriesConsistency(data[col], types[col])
        #print(col + " consistency is: %.2f%%"%current_const)
        consistent_data.append(current_const)
    average_const = sum(consistent_data) /len(consistent_data)
    return average_const

In [48]:
# We will calculate the consistency of all columns
for data in chg_datas:
    print('='*50)
    print('일련번호 : ', data['SerialNumber'][0])
    columns = data.columns
    const_dictionary = {}
    default_type = (float, int)
    numeric_type = (float, int)
    # If there is a special type to be checked, please add in this list.
    # In format: "Name_of_Column": (range_min, range_max)
    #For example,
    set_types = {
        "Voltage": numeric_type,
        "Current": numeric_type,
        "Power": numeric_type,
    }
    # Time and Date types will be set automatically
    # Columns names with Date and Time must be or start with the words Date and Time, respectively.
    for column_name in columns:
        if column_name in set_types:
            const_dictionary[column_name] = set_types[column_name]
        else:
            const_dictionary[column_name] = default_type
    print(f"일관성 품질지수 : {getConsistency(data, const_dictionary):.2f}")

일련번호 :  1000
일관성 품질지수 : 100.00
일련번호 :  1001
일관성 품질지수 : 100.00
일련번호 :  1002
일관성 품질지수 : 100.00
일련번호 :  1003
일관성 품질지수 : 100.00
일련번호 :  1004
일관성 품질지수 : 100.00
일련번호 :  1005
일관성 품질지수 : 100.00
일련번호 :  1006
일관성 품질지수 : 100.00
일련번호 :  1007
일관성 품질지수 : 100.00
일련번호 :  1008
일관성 품질지수 : 100.00
일련번호 :  1009
일관성 품질지수 : 100.00
일련번호 :  1010
일관성 품질지수 : 100.00
일련번호 :  1011
일관성 품질지수 : 100.00
일련번호 :  1012
일관성 품질지수 : 100.00
일련번호 :  1013
일관성 품질지수 : 100.00
일련번호 :  1014
일관성 품질지수 : 100.00
일련번호 :  1015
일관성 품질지수 : 100.00
일련번호 :  1016
일관성 품질지수 : 100.00
일련번호 :  1017
일관성 품질지수 : 100.00
일련번호 :  1018
일관성 품질지수 : 100.00
일련번호 :  1019
일관성 품질지수 : 100.00
일련번호 :  1020
일관성 품질지수 : 100.00
일련번호 :  1021
일관성 품질지수 : 100.00
일련번호 :  1022
일관성 품질지수 : 100.00
일련번호 :  1023
일관성 품질지수 : 100.00
일련번호 :  1024
일관성 품질지수 : 100.00
일련번호 :  1025
일관성 품질지수 : 100.00
일련번호 :  1026
일관성 품질지수 : 100.00
일련번호 :  1027
일관성 품질지수 : 100.00
일련번호 :  1028
일관성 품질지수 : 100.00
일련번호 :  1029
일관성 품질지수 : 100.00
일련번호 :  1030
일관성 품질지수 : 100.00
일련번호 :  1031
일관성 품질지수 : 100.00
일련번호 :  

▶ 정확성 품질 지수
$$
    정확성 품질지수 = 1- \left(\frac{\text{정확성에 위배되는 데이터의 개수}}{\text{전체 데이터의 개수}}\right) \times 100
$$
① 데이터 항목에 대하여 검증된 정확한 정보 (참조 데이터)가 있는지를 확인하고, 참조 데이터가 있는 경우, 데이터 항목 값과 참조 데이터의 값을 비교한다.  
② 여기에서 배터리 데이터는 참조 데이터가 존재하지 않으므로 정확성을 판단하지 않는다.

▶ 무결성 품질 지수
$$
   무결성 품질지수 = \left\{ 1 - \left(\frac{\text{유일성, 유효성, 일관성 중 100 \%가 아닌 지수}}{3}\right)  \right\} \times 100
$$
- 무결성 품질 지수는 앞에서 구한 유일성, 유효성, 일관성 지수를 기초로 위의 식으로 계산된다. 

▶ 가중치 지수
$$
    가중치 지수 = 품질 지수 \times 가중치
$$
- 데이터의 및 적용 업무의 특성에 따라 품질지수의 중요도가 달라지며, 이에 따라 적절한 가중치 지수를 부여한다.

▶ 데이터 품질지수
$$
    데이터 품질지수 = \sum 가중치지수
$$
① 앞에서 구한 6가지의 데이터 품질지수에 적절한 가중치를 반영하여 최종적으로 데이터 품질지수를 구한다. 